# Método Simplex 

O método simplex é um procedimento inventado no metade final dos anos 1940 por George Dantzig para resolver programas lineares que funciona iterativamente. Cada iteração, dentro do espaço viável, melhora a função objetivo. É um algoritmo robusto que resolve o problema dual simultaneamente. A partir de um programa linear qualquer, o primeiro passo é transformar para a forma canônica e nesse formato, o problema é resolvido de maneira simples. 

## Forma canônica 

Um problema linear com a seguinte estrutura: 

1. As variáveis de decisão são não negativas; 
2. Todas as restrições são em forma de equações; 
3. Os coeficientes do lado direito das restrições são não negativos; 
4. Em cada restrição, uma variável de decisão é isolada com coeficiente +1, enquanto nas outras restrições, essa variável não aparece. Ela também aparece com coeficiente zero na função objetivo. 

## Critérios para solução do problema 

Consideremos inicialmente o problema em sua forma canônica. Temos três importantes critérios: 

1. Critério da ilimitação: estabelece quando o problema é ilimitado ou não; 
2. Critério da melhora: estabelece quando e como melhorar a função objetivo a partir de uma variável não básica; 
3. Critério da razão e pivotamento: estabelece como alterar a base de variáveis.

In [1]:
import numpy as np
np.set_printoptions(precision=2)
from scipy.optimize import linprog

Estamos interessados em maximizar uma função objetivo linear sujeita a retrições lineares de igualdade e desigualdade. Dessa forma, resolvemos problemas do tipo: 

$$\begin{align}
\max_x \ & c^T x \\
\mbox{tal que}  \ & A_{ub} x \leq b_{ub}, \\
& A_{eq} x = b_{eq}, \\
& l \leq x \leq u
\end{align}$$

de forma que $x$ é o vetor de variáveis de decisão, $c$, $b_{ub}, b_{eq}, l, u$ são vetores e $A_{ub}$ e $A_{eq}$ são matrizes também. 

## Desenvolvendo o algoritmo 

O primeiro passo, que pode ser deixado do lado, é conferir as dimensões do sistema.

In [2]:
def _check_dimensions(self, c, A_ub, A_eq, b_ub, b_eq): 

    if A_ub.shape[1] != self.n_var: 
        raise Exception("The number of columns of A_ub must be the number of lines of c.")
    elif A_eq.shape[1] != self.n_var:
        raise Exception("The number of columns of A_eq must be the number of lines of c.")
    elif b_ub.shape[0] != A_ub.shape[0]:
        raise Exception("The number of lines of A_ub must be the number of lines of b_ub.")
    elif b_eq.shape[0] != A_eq.shape[0]: 
        raise Exception("The number of lines of A_eq must be the number of lines of b_eq.")

A partir da inserção dos valores pelo usuário, precisamos ajeitar os valores de forma que eles fiquem no formato canônico, como descrito acima. Separamos o processo na seguintes etapas. 

1. Convertemos os limites de caixa das variáveis em restrições de desigualdades. Por padrão, assumimos que as variáveis são não negativas. Restrições do tipo $x \le u$ são inseridas como desigualdades normalmente. Restrições do tipo $x \ge l$ são transformadas em $-x \le -l$. Por fim, caso $l = -\infty$, escrevemos $x = x^+ - x^-$ em todas as restrições e na função objetivo, de forma que $x^+, x^- \ge 0$. 

Observo que estamos criando duas variáveis novas que depois serão juntas novamente para formar $x$. 

In [51]:
def _bounds_handler(self, bounds, c, A_ub, A_eq, b_ub, b_eq):

    l = np.array([b[0] for b in bounds], dtype = np.float64)
    u = np.array([b[1] for b in bounds], dtype = np.float64)
    if len(bounds) == 0: 
        return (c, A_ub, A_eq, b_ub, b_eq)
    elif len(bounds) < self.n_var: 
        raise Exception("You need to specify dimension of c bounds.")
    else:
        col_bar = self.n_var + sum(l == -np.inf)
        lin_bar = A_ub.shape[0] + sum(l > -np.inf) + sum(u < np.inf)
        A_ub_bar = np.zeros((lin_bar, col_bar))
        A_eq_bar = np.zeros((A_eq.shape[0], col_bar))
        b_ub_bar = np.zeros((lin_bar, 1))
        b_ub_bar[:A_ub.shape[0],0:1] = b_ub
        c_bar = np.zeros((col_bar, 1))

        col = 0
        lin = 0
        for i in range(self.n_var):
            if l[i] == -np.inf: 
                A_ub_bar[:A_ub.shape[0], col] = A_ub[:,i]
                A_ub_bar[:A_ub.shape[0], col+1] = -A_ub[:,i]
                A_eq_bar[:A_eq.shape[0], col] = A_eq[:,i]
                A_eq_bar[:A_eq.shape[0], col+1] = -A_eq[:,i]
                c_bar[col] = c[i]
                c_bar[col+1] = -c[i]
                if u[i] < np.inf: 
                    A_ub_bar[A_ub.shape[0]+lin, col] = 1
                    A_ub_bar[A_ub.shape[0]+lin, col+1] = -1
                    b_ub_bar[A_ub.shape[0]+lin] = u[i]
                    lin += 1
                old_name = self.variables_names[col]
                self.variables_names[col] = old_name + "+1"
                self.variables_names.insert(col+1, old_name+ "-1")
                col += 2
            else: 
                A_ub_bar[:A_ub.shape[0], col] = A_ub[:,i]
                A_ub_bar[A_ub.shape[0]+lin, col] = -1 
                b_ub_bar[A_ub.shape[0]+lin] = -l[i]
                A_eq_bar[:A_eq.shape[0], col] = A_eq[:,i]
                c_bar[col] = c[i]
                if u[i] < np.inf: 
                    A_ub_bar[A_ub.shape[0]+lin+1, col] = 1
                    b_ub_bar[A_ub.shape[0]+lin+1] = u[i]
                    lin += 1
                lin += 1
                col += 1
    return (c_bar, A_ub_bar, A_eq_bar, b_ub_bar, b_eq)

2. Agora com as desigualdades em mãos, podemos adicionar as variáveis slack, quando $b$ é positivo na restrição $i$, e as variáveis surplus, quando $b$ é negativo. Na função, não diferenciamos os tipos de variáveis. Com a adição de variáveis slack, teremos um sistema de equações como restrições do modelo que chamaremos de $Ax = b$. 

In [65]:
def _add_slack_variables(self, A_ub, b_ub):

    slack_var = np.eye(A_ub.shape[0])
    return (slack_var, A_ub, b_ub)

3. Lembre que queremos que o vetor $b$ apenas com entradas positivas. Por isso uma função para forçar isso!

In [97]:
def _force_b_positive(self, slack_var, A, b): 
    
    negative_const = np.where(b < 0)[0]
    A[negative_const] = -A[negative_const]
    slack_var[negative_const] = - slack_var[negative_const]
    b[negative_const] = -b[negative_const]
    
    return slack_var, A, b

4. Nem todas as restrições têm uma variável isolada, isto é, com coeficiente +1 e que não apareça nas outras. Por esse motivo, precisamos adicionar as variáveis artificiais. Em particular, vamos adicioná-las para variáveis sem variável slack +1, isto, é, vamos olhar a matriz de variáveis slack das restrições e, para as restrições em que não haja valor positivo +1, vamos adicionar uma variável artificial. 

In [87]:
def _add_artificial_variables(self, slack, A, b, c): 

    surplus = np.where(slack==-1)[0]
    art_variables = np.where(slack.sum(axis = 1) <= 0)[0]

    slack_bar = np.eye(slack.shape[0], slack.shape[0]+len(surplus))
    slack_bar[surplus, slack.shape[0]:] = -np.eye(len(surplus))

    self.table = np.zeros((slack_bar.shape[0]+1, slack_bar.shape[1]+A.shape[1]+1))
    self.table[:-1, :] = np.hstack([slack_bar, A, b])
    self.table[-1, slack_bar.shape[1]:-1] = c.flatten()

    additional_variables = ["s"+str(i) for i in range(slack_bar.shape[1])]
    additional_variables.extend(self.variables_names)
    self.variables_names = list(additional_variables)        
    for i in art_variables:
        self.variables_names[i] = "a" + self.variables_names[i][1]

    return art_variables

Com isso, teremos a forma canônica e estamos prontos para começar a otimização. A otimização ocorre segundo o procedimento fase 1 - fase 2. **EXPLICAR ESSA PARTE.**

In [175]:
def _phase1(self, table, var): 

    new_objective = table[self.art_variables, :].sum(axis=0)
    table = np.vstack([table, new_objective])
    table[-1, self.art_variables] = 0.0
    table[-1,-1] = 0.0

    table, var = self._iteration(table, var, phase1 = True)
    
    if -table[-1,-1] < new_objective[-1]:
        return None, None
    else: 
        for i in range(table.shape[0]-2): 
            if var[i][0] == "a": 
                for j, v in enumerate(var):
                    if (v[0] != "a") & (table[i,j] != 0):
                        a_ij = table[i,j]
                        norm_line_i = table[i,:]/a_ij
                        table = table - np.outer(table[:,j], table[i,:])/a_ij
                        table[i,:] = norm_line_i
                        var[i], var[j] = var[j], var[i]
                        table[:,[i,j]] = table[:,[j,i]]                            
                        break
        # keep the artificial variables always zero. 
        var_fixed = list(var)
        art = list(range(len(var)+1))
        for j, v in enumerate(var_fixed):
            if v[0] == "a": 
                art.remove(j)
                var.remove(v)
        table = table[:-1,art]

    return table, var   

Na fase 2, todas as variáveis artificiais puderam ser removidas (dado que sempre valerão 0). A otimização ocorre normalmente. **EXPLICAR COMO OCORRE OTIMIZAÇÃO.**

In [176]:
def _phase2(self, table, var):

    return self._iteration(table, var)

def _iteration(self,tb, var, phase1=False): 

    while sum(tb[-1,:-1] > 0) > 0: 

        s = np.argmax(tb[-1,:-1])
        if sum(tb[:-1,s] > 0) == 0: 
            return None, None

        positive_a = np.where(tb[:-1-phase1,s] > 1e-16)[0]
        r = positive_a[(tb[positive_a,-1]/tb[positive_a,s]).argmin()]

        a_rs = tb[r,s]
        norm_line_r = tb[r,:]/a_rs
        tb = tb - np.outer(tb[:,s], tb[r,:])/a_rs
        tb[r,:] = norm_line_r

        # Change of variables
        var[r], var[s] = var[s], var[r]
        tb[:,[r,s]] = tb[:,[s,r]]

    return (tb, var) 

Vamos retornar, por fim, um objeto do tipo `OptimizeResult` que é descrito a seguir. Nesse caso `x` indica a solução ótima, `fun` o valor ótimo, `slack` os valores das variáveis slack e surplus, `sucess` se a otimização encerrou corretamente e `message` sobre o que aconteceu, caso algo tenha dado errado. 

In [177]:
class OptimizeResult: 
    
    def __init__(self, x, fun, slack, sucess, message): 
        
        self.x = x
        self.fun = fun
        self.slack = slack
        self.sucess = sucess
        self.message = message
        
        self.result = {"x": x, "fun": fun, "slack": slack, "success": sucess, "message": message}

A partir da tabela resultante do algoritmo, podemos calcular os resultados. 

In [178]:
def _results(self, tb, var): 

    fun = -tb[-1,-1]

    x = np.zeros(self.n_var)
    slack = np.array([0. for i in var if i[0] == 's'])
    for i, v in enumerate(var): 
        if v[0] == "s": 
            slack[int(v[1])] = 0. if i >= tb.shape[0]-1 else tb[i,-1]
        else:
            if len(v) == 2:
                x[int(v[1])] = 0. if i >= tb.shape[0]-1 else tb[i,-1]
            else:
                x[int(v[1])] += int(v[2:])*(0. if i >= tb.shape[0]-1 else tb[i,-1])

    return x, slack, fun

A função `optimize` é a chamada pelo usuário para realizar todo o procedimento. 

In [179]:
def optimize(self):

    tb = np.array(self.table)
    var = list(self.variables_names)

    tb, var = self._phase1(tb, var)
    if tb is None: 
        message = "The problem is infeasible."
        sucess = False
        fun = None
        x = None
        slack = None
    else: 
        tb, var = self._phase2(tb, var)
        if tb is None: 
            message = "The primal problem is unbounded."
            sucess = False
            fun = None
            x = None
            slack = None
        else: 
            message = "Optimization terminated successfully."
            sucess = True
            x, slack, fun = self._results(tb, var)

    return OptimizeResult(x, fun, slack, sucess, message)

Por fim, definimos a classe para o método simplex. 

In [180]:
class SimplexMethod: 
    
    def __init__(self, c, **kwargs): 
        
        c = c.reshape(-1,1)
        self.n_var = c.shape[0]
        A_ub = kwargs.get("A_ub", np.empty(shape=(0,self.n_var)))
        A_eq = kwargs.get("A_eq", np.empty(shape=(0,self.n_var)))
        b_ub = kwargs.get("b_ub", np.empty(shape=(0,1)))
        b_eq = kwargs.get("b_eq", np.empty(shape=(0,1)))
        # If bounds is an empty list, treating as [0, +inf].
        bounds = kwargs.get("bounds", [])
        
        b_ub = b_ub.reshape(-1,1)
        b_eq = b_eq.reshape(-1,1)
        
        self._check_dimensions(c, A_ub, A_eq, b_ub, b_eq)
        self.variables_names = ["x"+ str(i) for i in range(self.n_var)]
        
        c, A_ub, A_eq, b_ub, b_eq = self._bounds_handler(bounds, c, A_ub, A_eq, b_ub, b_eq)
        slack, A_ub, b_ub = self._add_slack_variables(A_ub, b_ub)
        
        # Joining all the equalities
        slack = np.vstack([slack, np.zeros((A_eq.shape[0], slack.shape[0]))])
        A = np.vstack([A_ub, A_eq])
        b = np.vstack([b_ub, b_eq])
        slack, A, b = self._force_b_positive(slack, A, b)
        self.art_variables = self._add_artificial_variables(slack, A, b, c)
        
SimplexMethod._check_dimensions = _check_dimensions
SimplexMethod._bounds_handler = _bounds_handler
SimplexMethod._add_slack_variables = _add_slack_variables
SimplexMethod._force_b_positive = _force_b_positive
SimplexMethod._add_artificial_variables = _add_artificial_variables
SimplexMethod._phase1 = _phase1
SimplexMethod._phase2 = _phase2
SimplexMethod._iteration = _iteration
SimplexMethod.optimize = optimize
SimplexMethod._results = _results

## Exemplos

Considere o problema 

$$
\begin{align}
\max_x \ & -3x_1 + x_3 \\
\mbox{tal que }  &x_1 + x_2 + x_3 + x_4 \le 4, \\
& -2x_1 + x_2 - x_3 = 1 \\
& 3x_2 + x_3 + x_4 = 9, \\
& x_1 \ge -3, x_2 \le 4, x_j \ge 0, j = 2,3,4 
\end{align}
$$

In [231]:
c = np.array([-3,0,1,0])
A_ub = np.array([[1,1,1,1]])
b_ub = np.array([4])
A_eq = np.array([[-2,1,-1,0], [0,3,1,1]])
b_eq = np.array([1, 9])
bounds = [[0, np.inf], [0,2.5], [0,np.inf], [0,np.inf]]

In [232]:
model = SimplexMethod(c, A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq, bounds = bounds)
res = model.optimize()
res.result

{'x': array([0. , 2.5, 1.5, 0. ]),
 'fun': 1.5,
 'slack': array([0. , 0. , 2.5, 0. , 1.5, 0. ]),
 'success': True,
 'message': 'Optimization terminated successfully.'}

In [233]:
res2 = linprog(-c, A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq, bounds = bounds, method = "simplex")

In [234]:
res2

     con: array([0., 0.])
     fun: -1.5
 message: 'Optimization terminated successfully.'
     nit: 7
   slack: array([0.])
  status: 0
 success: True
       x: array([0. , 2.5, 1.5, 0. ])